In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

drive.mount('/content/drive', force_remount=True)
data = pd.read_csv('/content/drive/My Drive/Диплом/data.csv')

Mounted at /content/drive


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Предобработка: приведение текста к нижнему регистру
data['queries'] = data['queries'].str.lower()
data['tags'] = data['tags'].str.lower()

# Создаём TF-IDF векторизатор
vectorizer = TfidfVectorizer()

# Объединяем запросы и теги в один набор для построения общего словаря TF-IDF
all_texts = pd.concat([data['queries'], data['tags']], axis=0)

# Обучаем векторизатор и трансформируем тексты в TF-IDF представление
tfidf_matrix = vectorizer.fit_transform(all_texts)

# Разделяем матрицу обратно на запросы и теги

query_vectors = tfidf_matrix[:len(data['queries'])]
tag_vectors = tfidf_matrix[len(data['queries']):]
# Вычисляем косинусное сходство для каждой пары (запрос, тег)
similarity_scores = [
    cosine_similarity(query_vectors[i], tag_vectors[i])[0, 0]
    for i in range(len(data))
]

# Добавляем релевантность в датасет
data['relevance'] = similarity_scores

# Посмотрим на первые несколько строк результата
data[['queries', 'tags', 'relevance']].head()


,queries,tags,relevance
0,how can i get chewy chocolate chip cookies?,baking cookies texture,0.208361
1,how should i cook bacon in an oven?,oven cooking-time bacon,0.461808
2,what is the difference between white and brown...,eggs,0.330939
3,what is the difference between baking soda and...,baking-soda baking-powder,0.741442
4,"in a tomato sauce recipe, how can i cut the ac...",sauce pasta tomatoes italian-cuisine,0.114587


In [ ]:
data.to_csv('/content/drive/My Drive/Диплом/updated_dataset.csv', index=False)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

!!! убрать нахрен, отсавить svc выволдить н резултатов (10), искать по косинусному в датасете и выводить с наибольшей релевантностью
# Преобразование тегов в формат мульти-лейбл
data['tags_list'] = data['tags'].str.split()  # Преобразуем строки тегов в списки

# Binarizer для мульти-лейбл задач
mlb = MultiLabelBinarizer()
tags_binary = mlb.fit_transform(data['tags_list'])

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    data['queries'], tags_binary, test_size=0.2, random_state=42
)

# Векторизация текстов запросов с помощью TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Ограничиваем количество фич
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Обучение модели (One-vs-Rest логистическая регрессия)
model = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
model.fit(X_train_tfidf, y_train)

# Предсказания на тестовых данных
y_pred = model.predict(X_test_tfidf)




/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 80 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 332 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 507 is present in all training examples.
  warnings.warn(


In [ ]:
# # Оценка качества модели
# report = classification_report(y_test, y_pred, target_names=mlb.classes_, output_dict=True)

# report_path = '/content/drive/My Drive/Диплом/classification_report.txt'

# # Сохраняем отчет в текстовый файл на Google Drive
# with open(report_path, 'w') as f:
#     f.write("Classification Report:\n")
#     for key in report:
#         f.write(f"{key}: {report[key]}\n")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/me

In [ ]:
# Оценка качества модели
from sklearn.metrics import classification_report

# Выводим отчёт по каждому тегу
report = classification_report(y_test, y_pred, target_names=mlb.classes_)
print(report)

# # Сохранение модели и векторизатора для использования
import joblib

joblib.dump(model, '/content/drive/My Drive/Диплом/папка/multilabel_model.pkl')
joblib.dump(tfidf_vectorizer, '/content/drive/My Drive/Диплом/папка/tfidf_vectorizer.pkl')
joblib.dump(mlb, '/content/drive/My Drive/Диплом/папка/label_binarizer.pkl')




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/me

                        precision    recall  f1-score   support

               acidity       0.00      0.00      0.00        11
             additives       0.00      0.00      0.00         2
               african       0.00      0.00      0.00         1
                 aging       0.00      0.00      0.00         2
               alcohol       0.00      0.00      0.00        17
       alcohol-content       0.00      0.00      0.00         1
               alfredo       0.00      0.00      0.00         3
            alkalinity       0.00      0.00      0.00         0
               allergy       0.00      0.00      0.00         8
                allium       0.00      0.00      0.00         0
           almond-milk       0.00      0.00      0.00         1
               almonds       0.00      0.00      0.00         3
     aluminum-cookware       0.00      0.00      0.00         2
         aluminum-foil       0.00      0.00      0.00         0
      american-cuisine       0.00      

['/content/drive/My Drive/Диплом/папка/label_binarizer.pkl']

In [ ]:
def predict_tags(query, model_path='/content/drive/My Drive/Диплом/папка/multilabel_model.pkl', vectorizer_path='/content/drive/My Drive/Диплом/папка/tfidf_vectorizer.pkl', binarizer_path='/content/drive/My Drive/Диплом/папка/label_binarizer.pkl'):
    # Загрузка модели и вспомогательных объектов
    model = joblib.load(model_path)
    vectorizer = joblib.load(vectorizer_path)
    binarizer = joblib.load(binarizer_path)

    # Преобразование запроса в TF-IDF вектор
    query_vector = vectorizer.transform([query.lower()])

    # Предсказание
    predicted_labels = model.predict(query_vector)

    # Преобразование предсказаний в текстовые теги
    tags = binarizer.inverse_transform(predicted_labels)
    return tags[0]

# Пример использования
query = "sauce spaghetti"
predicted_tags = predict_tags(query)
print("Predicted Tags:", predicted_tags)


Predicted Tags: ('sauce', 'spaghetti')


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sentence_transformers import SentenceTransformer, util

df = data

# Step 1: Tag Frequency Analysis
def calculate_tag_frequency(tags_list):
    tag_counts = {}
    for tags in tags_list:
        for tag in tags.split():
            tag_counts[tag] = tag_counts.get(tag, 0) + 1
    return tag_counts

tag_counts = calculate_tag_frequency(df['tags'])

def tag_frequency_score(tags, tag_counts):
    return sum(tag_counts[tag] for tag in tags.split())

df['TFS'] = df['tags'].apply(lambda x: tag_frequency_score(x, tag_counts))

# Step 2: Query Length Score
def query_length_score(query):
    length = len(query.split())
    if 5 <= length <= 10:
        return 1.0  # Optimal length
    elif length < 5:
        return 0.5  # Too short
    else:
        return 0.7  # Too long

df['QLS'] = df['queries'].apply(query_length_score)

# Step 3: Semantic Similarity Score (SSS)
model = SentenceTransformer('all-MiniLM-L6-v2')  # Load pre-trained model
popular_queries = [
    "how to bake cookies",
    "cooking times for meat",
    "safe food storage",
    "how to use spices",
    "basic cooking techniques"
]

popular_embeddings = model.encode(popular_queries, convert_to_tensor=True)

def semantic_similarity_score(query):
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarity = util.cos_sim(query_embedding, popular_embeddings).max().item()
    return similarity

df['SSS'] = df['queries'].apply(semantic_similarity_score)

# Step 4: Combine Scores
w1, w2, w3 = 0.4, 0.3, 0.3  # Weights for each component
df['Relevance'] = w1 * df['TFS'] + w2 * df['QLS'] + w3 * df['SSS']

# Normalize Relevance
scaler = MinMaxScaler()
df['Relevance'] = scaler.fit_transform(df[['Relevance']])

# Save the result
df.to_csv('relevance_dataset.csv', index=False)
print(df[['id', 'queries', 'Relevance']])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

          id                                            queries  Relevance
0          1        How can I get chewy chocolate chip cookies?   0.466042
1          2                How should I cook bacon in an oven?   0.143516
2          3  What is the difference between white and brown...   0.112521
3          4  What is the difference between baking soda and...   0.018683
4          5  In a tomato sauce recipe, how can I cut the ac...   0.238074
...      ...                                                ...        ...
15399  73670                          Poached eggs at altitude?   0.120295
15400  73678  How to thicken buttercream without adding more...   0.007135
15401  73680       Looking for old Italian recipe for chamellas   0.378413
15402  73681     Can i make ice cream with artificial sweetener   0.030713
15403  73683          What are these white 'bits' in my nutella   0.064273

[15404 rows x 3 columns]


In [ ]:
# """
# взять у даши косинусное расстояние
# переделать для датасета
# """
# !!!!!оставить только это (для каджого столбца отдельно), словарь убрать, если релеватность 0, то ставь 1 (по смыслу подходит, по словам - нет, потом подправим)
# !!! возможно, переделать на ллм